In [1]:
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model
import tensorflow as tf

In [2]:
model_version = '2'
dataset_version = '1'

train_ds = tf.data.Dataset.load(f'../dataset/V{dataset_version}/train_ds')
train_ds.cardinality().numpy()

np.int64(60000)

In [3]:
def model_v1():
    # Inputs 
    image_input = Input(shape=train_ds.element_spec[0].shape[:2], name='image_input')

    # Model structure
    f = Flatten(name='flatten_layer')(image_input)
    d = Dense(128, activation='relu', name='process_layer')(f)
    out_d = Dense(10, activation='softmax', name='output_layer')(d)

    # Assemble model
    model = Model(inputs=image_input, outputs=out_d, name=f'model_v{model_version}')
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [4]:
def model_v2():
    # Inputs 
    image_input = Input(shape=train_ds.element_spec[0].shape[:2], name='image_input')

    # Model structure
    f = Flatten(name='flatten_layer')(image_input)
    d = Dense(train_ds.element_spec[0].shape[0] * train_ds.element_spec[0].shape[1], activation='relu', name='process_layer')(f)
    #d = Dense(train_ds.element_spec[0].shape[0] * train_ds.element_spec[0].shape[1], activation='relu')(image_input)
    d = Dropout(0.2, name='dropout_layer')(d)
    d = Dense(10, activation='softmax', name='output_layer')(d)

    # Assemble model
    model = Model(inputs=image_input, outputs=d, name=f'model_v{model_version}')
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [5]:
model = model_v2()
model.summary()

Model: "model_v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 28, 28)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer (Flatten)         │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ process_layer (Dense)           │ (None, 784)            │       615,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer (Dropout)         │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │         7,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 623,290 (2.38 MB)

 Trainable params: 623,290 (2.38 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.save(f'V{model_version}/model.keras')